# 🇰🇪 Localized AI: Kenyan Bank Customer Support Classifier
**Hafsa Yahya Mohamud**

**Mini-Project for Local AI Application**

## Objective
To fine-tune a Large Language Model (DistilBERT) to understand Kenyan code-mixed text (Swahili, English, Sheng) and categorize customer support queries.

## Categories
1. **Billing:** Issues with tokens, meters, and KPLC.
2. **Reversals:** M-Pesa transactions to wrong numbers.
3. **Network:** Data bundles, internet speed, and coverage.
4. **General:** Inquiries about location, hours, etc.

In [3]:
# Install the SimpleTransformers library (wraps BERT/DistilBERT)
!pip install simpletransformers

## 3. Setup and Data Loading
We import necessary libraries and load our custom dataset (`customer_support_data.csv`). We also map our text categories to numbers (0-3) so the model can understand them.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.metrics import accuracy_score
import logging

# Setup logging to keep output clean
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Load the dataset
df = pd.read_csv("customer_support_data.csv")

# Map categories to numbers
label_map = {
    "billing_issue": 0,
    "transaction_reversal": 1,
    "network_issue": 2,
    "general_inquiry": 3
}
df['labels'] = df['label'].map(label_map)

# Split data: 80% for training, 20% for testing
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Data Loaded: {len(train_df)} training rows, {len(test_df)} test rows.")

Data Loaded: 79 training rows, 20 test rows.


## 4. Model Configuration
We are using **DistilBERT**, a lighter and faster version of the BERT Transformer model. We configure it to train for 5 epochs (loops) using the GPU.

In [6]:
# Configure the model
model_args = ClassificationArgs()
model_args.num_train_epochs = 5
model_args.train_batch_size = 16
model_args.overwrite_output_dir = True

# Initialize DistilBERT
model = ClassificationModel(
    "distilbert",
    "distilbert-base-uncased",
    num_labels=4,
    args=model_args,
    use_cuda=True  # Assumes you are using Colab GPU
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5. Training (Fine-Tuning)
We now train the model on our specific Kenyan dataset. This is where the "Local Adaptation" happens.

In [7]:
# Train the model
model.train_model(train_df)
print("Training Complete!")

0it [00:00, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:924: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:950: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


Running Epoch 2 of 5:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/5 [00:00<?, ?it/s]

Training Complete!


## 6. Evaluation
We test the model on the 20% of data it has never seen to calculate the Accuracy score.

In [8]:
# Evaluate model
result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_score)

print(f"Model Accuracy: {result['acc'] * 100:.2f}%")

0it [00:00, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Model Accuracy: 55.00%


/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:1570: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


## 7. Local Adaptation Test (Qualitative)
We test the model on new, unseen sentences containing heavy Sheng and local context to prove it works better than a generic model.

In [9]:
# Custom sentences with local slang
test_sentences = [
    "Manze tokens zangu zimeisha na mpesa haifanyi.",  # Should be Billing
    "Nimetuma pesa wrong number, please reverse.",      # Should be Reversal
    "Internet iko down sana leo.",                      # Should be Network
    "Mnafungua saa ngapi?"                              # Should be General
]

# Get predictions
predictions, raw_outputs = model.predict(test_sentences)

# Map numbers back to names
inv_map = {v: k for k, v in label_map.items()}

print("--- Prediction Results ---")
for text, pred_id in zip(test_sentences, predictions):
    print(f"Text: {text}")
    print(f"Prediction: {inv_map[pred_id]}")
    print("--------------------------")

0it [00:00, ?it/s]

Predicting:   0%|          | 0/1 [00:00<?, ?it/s]

--- Prediction Results ---
Text: Manze tokens zangu zimeisha na mpesa haifanyi.
Prediction: billing_issue
--------------------------
Text: Nimetuma pesa wrong number, please reverse.
Prediction: transaction_reversal
--------------------------
Text: Internet iko down sana leo.
Prediction: billing_issue
--------------------------
Text: Mnafungua saa ngapi?
Prediction: general_inquiry
--------------------------


/usr/local/lib/python3.12/dist-packages/simpletransformers/classification/classification_model.py:2260: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
